In [ ]:
!pip install q scipy==1.1.0
#use an earlier version of scipy to process sparse matrix operations easily
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable
# cuda = torch.device('cuda')     # Default CUDA device


     |████████████████████████████████| 31.2MB 164kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)), ])
trainset = datasets.MNIST('./',download = True, train = True, transform = transform)
testset = datasets.MNIST('./',download= True, train= False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

dataiter = iter(trainloader)
images, labels = dataiter.next()

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!




/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
#padding same 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, padding = (2,2))
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, padding = (2,2))
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(980, 50)
        self.fc2 = nn.Linear(50, 10)
        self.sc1 = nn.Conv2d(1,10,kernel_size = (1,1),stride = (1,1), bias = False)
        self.sc2 = nn.Conv2d(10,20,kernel_size = (1,1),stride = (1,1), bias = False)

    def forward(self, x):
        shortcut = x
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) + F.max_pool2d(self.sc1(x),2)

        shortcut = x
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) + F.max_pool2d(self.sc2(x),2)
        x = x.view(-1, 980)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
#set random seed for reproducibility
torch.manual_seed(0)
net = Net()
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
inpt = torch.zeros(64,1,28,28)
outpt = net(inpt)
outpt.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([64, 10])

In [ ]:
#training
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum = 0.9)
for epoch in range(12):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        # labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch Number: 1Training Accuracy: 77 %
Epoch Number: 2Training Accuracy: 91 %
Epoch Number: 3Training Accuracy: 94 %
Epoch Number: 4Training Accuracy: 95 %
Epoch Number: 5Training Accuracy: 95 %
Epoch Number: 6Training Accuracy: 96 %
Epoch Number: 7Training Accuracy: 96 %
Epoch Number: 8Training Accuracy: 96 %
Epoch Number: 9Training Accuracy: 96 %
Epoch Number: 10Training Accuracy: 96 %
Epoch Number: 11Training Accuracy: 97 %
Epoch Number: 12Training Accuracy: 97 %


In [ ]:
torch.save(net.state_dict(),'weights_f.pth')

In [ ]:
#testing #10 epochs
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_3.npy',acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 97 %


In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].resize(1,10*1*5*5))
wi2 = np.asarray(wi['conv2.weight'].resize(1,20*10*5*5))
wi3 = np.asarray(wi['fc1.weight'].resize(1,980*50))  #size of the resulting image after the given layers of convolutions 
wi4 = np.asarray(wi['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 
wi5 = np.asarray(wi['sc1.weight'].resize(1,10*1*1*1))
wi6 = np.asarray(wi['sc2.weight'].resize(1,20*10*1*1))
 
wi_vec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.00149736,  0.10728873, -0.16460903, ...,  0.00892442,
        0.11250275, -0.26631463], dtype=float32)

In [ ]:
#vectorize wf
wf =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].resize(1,10*1*5*5))
wf2 = np.asarray(wf['conv2.weight'].resize(1,20*10*5*5))
wf3 = np.asarray(wf['fc1.weight'].resize(1,980*50))  #size of the resulting image after the given layers of convolutions 
wf4 = np.asarray(wf['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 
wf5 = np.asarray(wf['sc1.weight'].resize(1,10*1*1*1))
wf6 = np.asarray(wf['sc2.weight'].resize(1,20*10*1*1))
 
wf_vec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]
wf_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([ 0.01744276,  0.01911643, -0.32000986, ..., -0.15053762,
        0.06669118, -0.34918207], dtype=float32)

In [ ]:
dim_w

54960

In [ ]:
#initialize A run 1
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 2
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0896

 Dist from Wi = 0.07287931166349663 , Dist from Wf =9.09295450122248

 n_rows = 4 , Accuracy =0.0896

 Dist from Wi = 0.08245608979311278 , Dist from Wf =9.092872700561731

 n_rows = 8 , Accuracy =0.0882

 Dist from Wi = 0.09324386982235029 , Dist from Wf =9.092768474789382

 n_rows = 16 , Accuracy =0.0879

 Dist from Wi = 0.11903901740016147 , Dist from Wf =9.092467358616648

 n_rows = 32 , Accuracy =0.0913

 Dist from Wi = 0.18592022945244327 , Dist from Wf =9.091345699239945

 n_rows = 64 , Accuracy =0.0907

 Dist from Wi = 0.28717883170473807 , Dist from Wf =9.088710649713478

 n_rows = 128 , Accuracy =0.0922

 Dist from Wi = 0.37655304389546795 , Dist from Wf =9.085446646212743

 n_rows = 256 , Accuracy =0.0916

 Dist from Wi = 0.560632359221631 , Dist from Wf =9.075947571113858

 n_rows = 512 , Accuracy =0.0941

 Dist from Wi = 0.7713540647476861 , Dist from Wf =9.060471613710677

 n_rows = 1024 , Accuracy =0.0948

 Dist from Wi = 1.14520130231169

In [ ]:
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
#initialize A + I run 1
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 2
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0906

 Dist from Wi = 0.09101951215586195 , Dist from Wf =9.09279101289523

 n_rows = 4 , Accuracy =0.0889

 Dist from Wi = 0.10964949137760208 , Dist from Wf =9.092585437862104

 n_rows = 8 , Accuracy =0.0916

 Dist from Wi = 0.16676228427768128 , Dist from Wf =9.09171729082437

 n_rows = 16 , Accuracy =0.0916

 Dist from Wi = 0.2971946394315799 , Dist from Wf =9.088388652714414

 n_rows = 32 , Accuracy =0.0959

 Dist from Wi = 0.5065382610260112 , Dist from Wf =9.079127267667907

 n_rows = 64 , Accuracy =0.0951

 Dist from Wi = 0.6242529162988462 , Dist from Wf =9.071793717351376

 n_rows = 128 , Accuracy =0.0979

 Dist from Wi = 0.9885948738838071 , Dist from Wf =9.039348047262916

 n_rows = 256 , Accuracy =0.0972

 Dist from Wi = 1.2517019298011258 , Dist from Wf =9.006685029185656

 n_rows = 512 , Accuracy =0.0998

 Dist from Wi = 1.3727287635166998 , Dist from Wf =8.989034913803577

 n_rows = 1024 , Accuracy =0.1032

 Dist from Wi = 1.6326443308679106 ,

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
#initialize I run 1
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768,250,250+20*250,250+20*250+980*50,250+20*250+980*50+50*10])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

wfcnnvec = np.concatenate((wf1,wf2,wf3,wf4),axis =1).T
wicnnvec = np.concatenate((wi1,wi2,wi3,wi4),axis =1).T 

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros(wfcnnvec.shape)
      w[0:n_rows] = wfcnnvec[0:n_rows]
      w[n_rows:] = wicnnvec[n_rows:]
      
      w = np.concatenate((w.T,wf5,wf6),axis = 1).T


      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0969

 Dist from Wi = 2.354392753088989 , Dist from Wf =8.783163878664732

 n_rows = 4 , Accuracy =0.0976

 Dist from Wi = 2.3626840938432063 , Dist from Wf =8.780937126980193

 n_rows = 8 , Accuracy =0.0907

 Dist from Wi = 2.3900262833608057 , Dist from Wf =8.773534482756844

 n_rows = 16 , Accuracy =0.0921

 Dist from Wi = 2.487514923688728 , Dist from Wf =8.746393683096178

 n_rows = 32 , Accuracy =0.0959

 Dist from Wi = 2.7095210116894046 , Dist from Wf =8.680185991238465

 n_rows = 64 , Accuracy =0.0946

 Dist from Wi = 2.7734364467213592 , Dist from Wf =8.659975937107243

 n_rows = 128 , Accuracy =0.1214

 Dist from Wi = 3.392927363931341 , Dist from Wf =8.436538203455536

 n_rows = 250 , Accuracy =0.12

 Dist from Wi = 3.834216647305743 , Dist from Wf =8.24535721826546

 n_rows = 256 , Accuracy =0.1187

 Dist from Wi = 3.835560243628726 , Dist from Wf =8.244732292365445

 n_rows = 512 , Accuracy =0.116

 Dist from Wi = 3.902116594395965 , Dist from W

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
#initialize Irev run 1
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768,250,250+20*250,250+20*250+980*50,250+20*250+980*50+50*10])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

wfcnnvec = np.concatenate((wf1,wf2,wf3,wf4),axis =1).T
wicnnvec = np.concatenate((wi1,wi2,wi3,wi4),axis =1).T 

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros(wfcnnvec.shape)
      w[0:-n_rows] = wicnnvec[0:-n_rows]
      w[-n_rows:] = wfcnnvec[-n_rows:]
      
      w = np.concatenate((w.T,wf5,wf6),axis = 1).T


      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0959

 Dist from Wi = 2.374780015537632 , Dist from Wf =8.77767354331898

 n_rows = 4 , Accuracy =0.0978

 Dist from Wi = 2.38222051961493 , Dist from Wf =8.775657146402452

 n_rows = 8 , Accuracy =0.1006

 Dist from Wi = 2.4092209766718375 , Dist from Wf =8.7682830269582

 n_rows = 16 , Accuracy =0.0979

 Dist from Wi = 2.4358897450037884 , Dist from Wf =8.760911716566048

 n_rows = 32 , Accuracy =0.097

 Dist from Wi = 2.5150301012623353 , Dist from Wf =8.738521416408112

 n_rows = 64 , Accuracy =0.0956

 Dist from Wi = 2.6219929672674454 , Dist from Wf =8.707025085233127

 n_rows = 128 , Accuracy =0.1027

 Dist from Wi = 2.837039282623712 , Dist from Wf =8.639348416641663

 n_rows = 250 , Accuracy =0.1036

 Dist from Wi = 3.361552312899202 , Dist from Wf =8.449088649252113

 n_rows = 256 , Accuracy =0.1055

 Dist from Wi = 3.396416813182575 , Dist from Wf =8.435134011170753

 n_rows = 512 , Accuracy =0.1351

 Dist from Wi = 4.15870313380212 , Dist from Wf 

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
from google.colab import files
files.download('accuracy_UI.npy') 
files.download('distance_i_UI.npy')
files.download('distance_f_UI.npy')

files.download('accuracy_I.npy') 
files.download('distance_i_I.npy')
files.download('distance_f_I.npy')

files.download('accuracy_AI.npy') 
files.download('distance_i_AI.npy')
files.download('distance_f_AI.npy')

files.download('accuracy_A.npy') 
files.download('distance_i_A.npy')
files.download('distance_f_A.npy')

In [ ]:
#initialize A run 1
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0938

 Dist from Wi = 0.016521575303589667 , Dist from Wf =9.016159235528495

 n_rows = 4 , Accuracy =0.0942

 Dist from Wi = 0.022950313623786696 , Dist from Wf =9.016145163314565

 n_rows = 8 , Accuracy =0.0964

 Dist from Wi = 0.03401849224809083 , Dist from Wf =9.016110195923773

 n_rows = 16 , Accuracy =0.0952

 Dist from Wi = 0.09883667839303173 , Dist from Wf =9.015632625270047

 n_rows = 32 , Accuracy =0.0934

 Dist from Wi = 0.16935142141225765 , Dist from Wf =9.014583762935466

 n_rows = 64 , Accuracy =0.0937

 Dist from Wi = 0.26572603009922074 , Dist from Wf =9.012257763699054

 n_rows = 128 , Accuracy =0.092

 Dist from Wi = 0.5056293128320412 , Dist from Wf =9.001985299558346

 n_rows = 256 , Accuracy =0.0953

 Dist from Wi = 0.6618800560460862 , Dist from Wf =8.991847147675783

 n_rows = 512 , Accuracy =0.0967

 Dist from Wi = 0.9799694115768837 , Dist from Wf =8.962759636568611

 n_rows = 1024 , Accuracy =0.0976

 Dist from Wi = 1.299308374688

In [ ]:
#initialize A run 3
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1034

 Dist from Wi = 0.015824275491295444 , Dist from Wf =9.135057233785503

 n_rows = 4 , Accuracy =0.1036

 Dist from Wi = 0.0417455749472116 , Dist from Wf =9.13497555438447

 n_rows = 8 , Accuracy =0.1039

 Dist from Wi = 0.0580375362258481 , Dist from Wf =9.134886573825304

 n_rows = 16 , Accuracy =0.1041

 Dist from Wi = 0.13328006333599277 , Dist from Wf =9.134098614388892

 n_rows = 32 , Accuracy =0.1068

 Dist from Wi = 0.2094497684668451 , Dist from Wf =9.132669481957878

 n_rows = 64 , Accuracy =0.1043

 Dist from Wi = 0.33346761545998566 , Dist from Wf =9.12898244184797

 n_rows = 128 , Accuracy =0.1008

 Dist from Wi = 0.4390755876706574 , Dist from Wf =9.12451279262769

 n_rows = 256 , Accuracy =0.108

 Dist from Wi = 0.6132329150615075 , Dist from Wf =9.11446468340027

 n_rows = 512 , Accuracy =0.1077

 Dist from Wi = 0.8744397161517878 , Dist from Wf =9.093122469983953

 n_rows = 1024 , Accuracy =0.1191

 Dist from Wi = 1.2745762364515536 , D

In [ ]:
np.save("accuracy_3.npy", acc)
np.save("distance_i_3.npy", dist_i)
np.save("distance_f_3.npy", dist_f)

In [ ]:
#initialize A + I run 1
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1058

 Dist from Wi = 0.055405547770996696 , Dist from Wf =9.130192421463317

 n_rows = 4 , Accuracy =0.1064

 Dist from Wi = 0.12863740011141897 , Dist from Wf =9.129454301698752

 n_rows = 8 , Accuracy =0.1048

 Dist from Wi = 0.188709847575658 , Dist from Wf =9.128410158453837

 n_rows = 16 , Accuracy =0.1019

 Dist from Wi = 0.31195081699319627 , Dist from Wf =9.125029869133558

 n_rows = 32 , Accuracy =0.1053

 Dist from Wi = 0.46748428121619895 , Dist from Wf =9.118384828065887

 n_rows = 64 , Accuracy =0.1082

 Dist from Wi = 0.7034993798044785 , Dist from Wf =9.103217675643902

 n_rows = 128 , Accuracy =0.1085

 Dist from Wi = 0.9457180379391358 , Dist from Wf =9.081249959161482

 n_rows = 256 , Accuracy =0.1087

 Dist from Wi = 1.3140290776464856 , Dist from Wf =9.035309126178912

 n_rows = 512 , Accuracy =0.1104

 Dist from Wi = 1.4601727139835494 , Dist from Wf =9.012845226156243

 n_rows = 1024 , Accuracy =0.1247

 Dist from Wi = 1.708017163336353

In [ ]:
#initialize A + I run 2
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0955

 Dist from Wi = 0.0728395406802102 , Dist from Wf =9.015880141415229

 n_rows = 4 , Accuracy =0.0939

 Dist from Wi = 0.0799974468596428 , Dist from Wf =9.015819470883631

 n_rows = 8 , Accuracy =0.0933

 Dist from Wi = 0.12722277137751878 , Dist from Wf =9.015276739486046

 n_rows = 16 , Accuracy =0.0922

 Dist from Wi = 0.17396500997828632 , Dist from Wf =9.01449590926181

 n_rows = 32 , Accuracy =0.0981

 Dist from Wi = 0.3101027170755118 , Dist from Wf =9.010839951411384

 n_rows = 64 , Accuracy =0.0959

 Dist from Wi = 0.43580219444495505 , Dist from Wf =9.00563583387452

 n_rows = 128 , Accuracy =0.0923

 Dist from Wi = 0.7279814497010683 , Dist from Wf =8.986737079174809

 n_rows = 256 , Accuracy =0.097

 Dist from Wi = 1.1120195588458386 , Dist from Wf =8.947335515294911

 n_rows = 512 , Accuracy =0.0977

 Dist from Wi = 1.2856215182353874 , Dist from Wf =8.924044914322439

 n_rows = 1024 , Accuracy =0.0982

 Dist from Wi = 1.5546102759903697 , 

In [ ]:
#initialize A + I run 3
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.106

 Dist from Wi = 0.07593761729091467 , Dist from Wf =9.134755308741555

 n_rows = 4 , Accuracy =0.1025

 Dist from Wi = 0.23404650931435164 , Dist from Wf =9.13207223491798

 n_rows = 8 , Accuracy =0.1008

 Dist from Wi = 0.28907898120907544 , Dist from Wf =9.130495847132488

 n_rows = 16 , Accuracy =0.1039

 Dist from Wi = 0.33397052325942245 , Dist from Wf =9.128964057385538

 n_rows = 32 , Accuracy =0.1008

 Dist from Wi = 0.4654296096561293 , Dist from Wf =9.123206473330875

 n_rows = 64 , Accuracy =0.11

 Dist from Wi = 0.6036312037303352 , Dist from Wf =9.115105619012214

 n_rows = 128 , Accuracy =0.1045

 Dist from Wi = 0.9315874555576104 , Dist from Wf =9.087445509889093

 n_rows = 256 , Accuracy =0.1019

 Dist from Wi = 1.1988820817335564 , Dist from Wf =9.056058902056781

 n_rows = 512 , Accuracy =0.1074

 Dist from Wi = 1.3656406871801106 , Dist from Wf =9.032416431389162

 n_rows = 1024 , Accuracy =0.1198

 Dist from Wi = 1.617610998370335 , D

In [ ]:
np.save("accuracy_AI3.npy", acc)
np.save("distance_i_AI3.npy", dist_i)
np.save("distance_f_AI3.npy", dist_f)

In [ ]:
#initialize I run 1
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1008

 Dist from Wi = 0.1538771977854263 , Dist from Wf =9.129063765557477

 n_rows = 4 , Accuracy =0.1083

 Dist from Wi = 0.19625445621339616 , Dist from Wf =9.128251071044724

 n_rows = 8 , Accuracy =0.1066

 Dist from Wi = 0.4131449620610192 , Dist from Wf =9.121008423849656

 n_rows = 16 , Accuracy =0.1057

 Dist from Wi = 0.6221993751902877 , Dist from Wf =9.109135598424864

 n_rows = 32 , Accuracy =0.1063

 Dist from Wi = 0.9683623263046544 , Dist from Wf =9.07886324507799

 n_rows = 64 , Accuracy =0.1032

 Dist from Wi = 1.4484780703310618 , Dist from Wf =9.014732093779823

 n_rows = 128 , Accuracy =0.1049

 Dist from Wi = 2.276076951830629 , Dist from Wf =8.842112711484885

 n_rows = 256 , Accuracy =0.0961

 Dist from Wi = 3.1977402841997997 , Dist from Wf =8.552072294607795

 n_rows = 512 , Accuracy =0.0914

 Dist from Wi = 3.2744246022616794 , Dist from Wf =8.523005747991032

 n_rows = 1024 , Accuracy =0.0935

 Dist from Wi = 3.440639311187397 , Di

In [ ]:
#initialize I run 2
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0938

 Dist from Wi = 0.07852273386996272 , Dist from Wf =9.015832435395067

 n_rows = 4 , Accuracy =0.0932

 Dist from Wi = 0.10803294943529311 , Dist from Wf =9.01552711740653

 n_rows = 8 , Accuracy =0.0961

 Dist from Wi = 0.226054716353175 , Dist from Wf =9.013340090558586

 n_rows = 16 , Accuracy =0.0952

 Dist from Wi = 0.32331510050655854 , Dist from Wf =9.010375556361906

 n_rows = 32 , Accuracy =0.0967

 Dist from Wi = 0.6730127219463744 , Dist from Wf =8.991020754930439

 n_rows = 64 , Accuracy =0.0926

 Dist from Wi = 1.2395216004527707 , Dist from Wf =8.930564737701049

 n_rows = 128 , Accuracy =0.0912

 Dist from Wi = 1.9781463674649522 , Dist from Wf =8.79649573933324

 n_rows = 256 , Accuracy =0.0902

 Dist from Wi = 3.114881073664878 , Dist from Wf =8.461023355919355

 n_rows = 512 , Accuracy =0.0882

 Dist from Wi = 3.2096737311802803 , Dist from Wf =8.425520450848637

 n_rows = 1024 , Accuracy =0.0928

 Dist from Wi = 3.340759538587456 , Di

In [ ]:
#initialize I run 3
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1054

 Dist from Wi = 0.30173921951659627 , Dist from Wf =9.130086227177117

 n_rows = 4 , Accuracy =0.107

 Dist from Wi = 0.47882927205677167 , Dist from Wf =9.122513009061995

 n_rows = 8 , Accuracy =0.1001

 Dist from Wi = 0.6880706654866081 , Dist from Wf =9.109120694328364

 n_rows = 16 , Accuracy =0.1052

 Dist from Wi = 0.8534816595396241 , Dist from Wf =9.095113529323571

 n_rows = 32 , Accuracy =0.1021

 Dist from Wi = 1.1559294420013826 , Dist from Wf =9.061641583883034

 n_rows = 64 , Accuracy =0.1021

 Dist from Wi = 1.445486261149887 , Dist from Wf =9.019982845429908

 n_rows = 128 , Accuracy =0.1104

 Dist from Wi = 2.417427052764621 , Dist from Wf =8.809402225101014

 n_rows = 256 , Accuracy =0.1081

 Dist from Wi = 3.0604494067692998 , Dist from Wf =8.607158100712995

 n_rows = 512 , Accuracy =0.1093

 Dist from Wi = 3.151552944647277 , Dist from Wf =8.574219216788151

 n_rows = 1024 , Accuracy =0.1071

 Dist from Wi = 3.23522353616482 , Dist

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
#IUI experiment 

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable
# cuda = torch.device('cuda')     # Default CUDA device


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)), ])
trainset = datasets.MNIST('./',download = True, train = True, transform = transform)
testset = datasets.MNIST('./',download= True, train= False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

dataiter = iter(trainloader)
images, labels = dataiter.next()

In [ ]:
#padding same 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, padding = (2,2))
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, padding = (2,2))
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(980, 50)
        self.fc2 = nn.Linear(50, 10)
        self.sc1 = nn.Conv2d(1,10,kernel_size = (1,1),stride = (1,1), bias = False)
        self.sc2 = nn.Conv2d(10,20,kernel_size = (1,1),stride = (1,1), bias = False)

    def forward(self, x):
        shortcut = x
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) + F.max_pool2d(self.sc1(x),2)

        shortcut = x
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) + F.max_pool2d(self.sc2(x),2)
        x = x.view(-1, 980)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

net = Net()
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
#training
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum = 0.9)
for epoch in range(12):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        # labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch Number: 1Training Accuracy: 76 %
Epoch Number: 2Training Accuracy: 91 %
Epoch Number: 3Training Accuracy: 94 %
Epoch Number: 4Training Accuracy: 95 %
Epoch Number: 5Training Accuracy: 95 %
Epoch Number: 6Training Accuracy: 96 %
Epoch Number: 7Training Accuracy: 96 %
Epoch Number: 8Training Accuracy: 96 %
Epoch Number: 9Training Accuracy: 96 %
Epoch Number: 10Training Accuracy: 97 %
Epoch Number: 11Training Accuracy: 97 %
Epoch Number: 12Training Accuracy: 97 %


In [ ]:
#testing #10 epochs
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_3.npy',acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 97 %


In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].resize(1,10*1*5*5))
wi2 = np.asarray(wi['conv2.weight'].resize(1,20*10*5*5))
wi3 = np.asarray(wi['fc1.weight'].resize(1,980*50))  #size of the resulting image after the given layers of convolutions 
wi4 = np.asarray(wi['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 
wi5 = np.asarray(wi['sc1.weight'].resize(1,10*1*1*1))
wi6 = np.asarray(wi['sc2.weight'].resize(1,20*10*1*1))

wicnnvec = np.concatenate((wi1,wi2,wi3,wi4),axis =1).T 
wi_vec = np.concatenate((wi1,wi2,wi3,wi4,wi5,wi6),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([ 0.03723502, -0.18686366, -0.08202501, ...,  0.18047878,
       -0.11825944,  0.09970215], dtype=float32)

In [ ]:
#vectorize wi
wf =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].resize(1,10*1*5*5))
wf2 = np.asarray(wf['conv2.weight'].resize(1,20*10*5*5))
wf3 = np.asarray(wf['fc1.weight'].resize(1,980*50))  #size of the resulting image after the given layers of convolutions 
wf4 = np.asarray(wf['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 
wf5 = np.asarray(wf['sc1.weight'].resize(1,10*1*1*1))
wf6 = np.asarray(wf['sc2.weight'].resize(1,20*10*1*1))
 
wfcnnvec = np.concatenate((wf1,wf2,wf3,wf4),axis =1).T
wf_vec = np.concatenate((wf1,wf2,wf3,wf4,wf5,wf6),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]
wf_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([ 0.14751475, -0.29752398, -0.39872152, ...,  0.03195131,
       -0.18476523, -0.00511502], dtype=float32)

In [ ]:
#initialize I run 1
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 32768,250,250+20*250,250+20*250+980*50,250+20*250+980*50+50*10])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros(wfcnnvec.shape)
      w[0:n_rows] = wfcnnvec[0:n_rows]
      w[n_rows:] = wicnnvec[n_rows:]
      
      w = np.concatenate((w.T,wf5,wf6),axis = 1).T


      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1048

 Dist from Wi = 2.6778160419460506 , Dist from Wf =8.697541023659197

 n_rows = 4 , Accuracy =0.0999

 Dist from Wi = 2.713696206442683 , Dist from Wf =8.686413040598339

 n_rows = 8 , Accuracy =0.0992

 Dist from Wi = 2.7784833129871003 , Dist from Wf =8.665907286152471

 n_rows = 16 , Accuracy =0.0971

 Dist from Wi = 2.8811115366318325 , Dist from Wf =8.632329634937793

 n_rows = 32 , Accuracy =0.1017

 Dist from Wi = 2.9752727857934524 , Dist from Wf =8.600329671754167

 n_rows = 64 , Accuracy =0.1021

 Dist from Wi = 3.01599199098928 , Dist from Wf =8.58613480694467

 n_rows = 128 , Accuracy =0.103

 Dist from Wi = 3.4245405187607343 , Dist from Wf =8.43151474220998

 n_rows = 250 , Accuracy =0.1131

 Dist from Wi = 4.116536504200372 , Dist from Wf =8.116159548845994

 n_rows = 256 , Accuracy =0.1074

 Dist from Wi = 4.116934595348332 , Dist from Wf =8.115957623741163

 n_rows = 512 , Accuracy =0.1125

 Dist from Wi = 4.206655279115067 , Dist from 

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
#initialize I run 1
from scipy import sparse
I = sparse.eye(dim_w) 
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 3276,250,250+20*250,250+20*250+980*50,250+20*250+980*50+50*10])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros(wfcnnvec.shape)
      w[0:-n_rows] = wicnnvec[0:-n_rows]
      w[-n_rows:] = wfcnnvec[-n_rows:]
      
      w = np.concatenate((w.T,wf5,wf6),axis = 1).T


      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  980*50]
      w3 = torch.from_numpy(w3).reshape(50,980)
      w4 = w[10*1*5*5+20*10*5*5 +  980*50: 10*1*5*5+20*10*5*5 +  980*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      w5 = w[10*1*5*5+20*10*5*5+980*50+10*50 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1]
      w5 = torch.from_numpy(w5).reshape(10,1,1,1)
      w6 = w[10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 : 10*1*5*5+20*10*5*5 +  980*50 + 10*50 + 10*1 + 20*10]
      w6 = torch.from_numpy(w6).reshape(20,10,1,1)

      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      wf['sc1.weight'] = (w5)
      wf['sc2.weight'] = (w6)

      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 


outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.0997

 Dist from Wi = 2.6840332382061076 , Dist from Wf =8.695624427776485

 n_rows = 4 , Accuracy =0.0948

 Dist from Wi = 2.6979667829636633 , Dist from Wf =8.69131139994207

 n_rows = 8 , Accuracy =0.0971

 Dist from Wi = 2.7127232517336615 , Dist from Wf =8.686716938650749

 n_rows = 16 , Accuracy =0.1026

 Dist from Wi = 2.7573395371707723 , Dist from Wf =8.672658029087343

 n_rows = 32 , Accuracy =0.111

 Dist from Wi = 2.8363233732053925 , Dist from Wf =8.647149144969518

 n_rows = 64 , Accuracy =0.1094

 Dist from Wi = 2.9414551275079273 , Dist from Wf =8.61195450206256

 n_rows = 128 , Accuracy =0.1124

 Dist from Wi = 3.1981449201877896 , Dist from Wf =8.519964065781927

 n_rows = 250 , Accuracy =0.1126

 Dist from Wi = 3.7037752203146397 , Dist from Wf =8.312639035235533

 n_rows = 256 , Accuracy =0.1196

 Dist from Wi = 3.7199429837631732 , Dist from Wf =8.305416474222676

 n_rows = 512 , Accuracy =0.1336

 Dist from Wi = 4.491032763983593 , Dist 

In [ ]:
np.save("accuracy_UI3.npy", acc)
np.save("distance_i_UI3.npy", dist_i)
np.save("distance_f_UI3.npy", dist_f)